In [1]:
# load the bokeh software and pydap client to access data
from bokeh.plotting import figure,output_file, show
from bokeh.layouts import column
from bokeh.io import output_notebook
from pydap.client import open_url
from bokeh.models import Range1d, LinearAxis, Label
import pandas as pd
import numpy as np
import seawater

In [2]:
#Is there a way to not hardwire the deployment date/directory into this?\
# unfortunately this may be the only way to do this as date of deployment is not predictable
deploydir='201907'
metdeploydate='20190731'
radfluxdate='20190729'

In [3]:
# standard depths on microcats
ctds=['0001','0010','0020','0040','0060','0080','0100','0150','0200','0250','0300']
# create iterator of number of microcats
ki=np.arange(0,len(ctds))
# create url from which to load data
urlpre='http://dods.mbari.org:80/opendap/data/ssdsdata/deployments/m1/'+deploydir+'/m1_ctd'
dsctd=[]
# load dataset id information into a list for access later.
# pretty cool how python treats this as an object and doesn't care about sticking it into a list
for ctd in ki:
    dataurl=urlpre+ctds[ctd]+'_'+radfluxdate+'_original.nc'
    dsctd.append(open_url(dataurl))

In [4]:
# create a count index to go back a week for hourly and 10 minute data
# not used for profile plot but for time-series plot (May extend this to work for 2 months if data is not too large)
week1=(7*24*60)/10
week2=(7*24)
# create a offset to convert times from seconds since to datetime values
# again this used for time-series plots and not the profile plot
offset=pd.datetime(1970,1,1)
# Color array for time-series plot (these are the colors used by the old matlab plots)
colors=['magenta','red','orange','green','cyan','blue']
# to do the upper water column and lower water column split into two pieces.
kj=np.arange(0,6)
# first bokeh setup create a tool tip for the temperature time-series plot and display the temperature value
TOOLTIPST=[(("Temperature"),"$y")]
# create the figure "handle" for the temperature time-series plot
ptemp=figure(plot_width=800,plot_height=250,title='M1 Temperature',
             x_axis_type='datetime',x_axis_label='time',y_axis_label='Deg C',outline_line_color='black',tooltips=TOOLTIPST)
# Do similarly for Salinty 
TOOLTIPSS=[(("Salinity"),"$y")]
psalt=figure(plot_width=800,plot_height=250,title='M1 Salinity',
             x_axis_type='datetime',x_axis_label='time',y_axis_label='psu',outline_line_color='black',tooltips=TOOLTIPSS)
#TOOLPROF=[(("Profile"),"$y")]
#pprof=figure(plot_width=800,plot_height=250,title='M1',
#             x_axis_label='Temp',y_axis_label='Depth',outline_line_color='black',tooltips=TOOLPROF)
# create empty lists for profile data for the profile plot
profiletemp=[]
profilesalt=[]
profiledep=[]
# okay now loop through the first 6 microcats and get time, depth, temp, and salinity
for ctd in kj:
    ctemp=dsctd[ctd]['Temperature']
    csalt=dsctd[ctd]['Salinity']
    pres=dsctd[ctd]['NominalDepth']
    cesecs=dsctd[ctd]['esecs']
    # use if 10 minute data
    if ctd < 3:
        temp=ctemp['Temperature'][-int(week1):-1]
        salt=csalt['Salinity'][-int(week1):-1]
        #pres=cpres['NominalDepth']
        etime=cesecs[-int(week1):-1]
    # use if hourly data
    else:
        temp=ctemp['Temperature'][-int(week2):-1]
        salt=csalt['Salinity'][-int(week2):-1]
        #pres=cpres['NominalDepth']
        etime=cesecs[-int(week2):-1]
    # remove singleton dimensions
    temp=np.squeeze(temp)
    salt=np.squeeze(salt)
    pres=np.squeeze(pres)
    # blank bad data to make plots "prettier"
    ll=temp < 0
    temp[ll]=np.nan
    ll=salt < 30
    salt[ll]=np.nan
    jnk=pd.to_timedelta(etime,unit='s')
    ztime=offset+jnk
    # get the most recent value for profile plot (however it diregards time, so it could be off)
    profiletemp.append(temp[-1])
    profilesalt.append(salt[-1])
    profiledep.append(pres)
    # plot the time-series temperature data as both a line and dots on the line
    ptemp.x_range=Range1d(ztime[0],ztime[-1])
    ptemp.line(ztime,temp,line_width=2,color=colors[ctd])
    ptemp.circle(ztime,temp,color=colors[ctd])
    # plot the salinity data in another plot
    psalt.x_range=Range1d(ztime[0],ztime[-1])
    psalt.line(ztime,salt,line_width=2,color=colors[ctd])
    psalt.circle(ztime,salt,color=colors[ctd])
    
# create the time-series plots for the lower microcat depths
TOOLTIPST2=[(("Temperature"),"$y")]
ptemp2=figure(plot_width=800,plot_height=250,title='M1 Temperature',
             x_axis_type='datetime',x_axis_label='time',y_axis_label='Deg C',outline_line_color='black',tooltips=TOOLTIPST2)
TOOLTIPSS2=[(("Salinity"),"$y")]
psalt2=figure(plot_width=800,plot_height=250,title='M1 Salinity',
             x_axis_type='datetime',x_axis_label='time',y_axis_label='psu',outline_line_color='black',tooltips=TOOLTIPSS2)
kl=np.arange(6,11)
for ctd in kl:
    ctemp=dsctd[ctd]['Temperature']
    csalt=dsctd[ctd]['Salinity']
    pres=dsctd[ctd]['NominalDepth']
    cesecs=dsctd[ctd]['esecs']
    if ctd < 3:
        temp=ctemp['Temperature'][-int(week1):-1]
        salt=csalt['Salinity'][-int(week1):-1]
        etime=cesecs[-int(week1):-1]
    else:
        temp=ctemp['Temperature'][-int(week2):-1]
        salt=csalt['Salinity'][-int(week2):-1]
        etime=cesecs[-int(week2):-1]
    temp=np.squeeze(temp)
    salt=np.squeeze(salt)
    pres=np.squeeze(pres)
    ll=temp < 0
    temp[ll]=np.nan
    ll=salt < 30
    salt[ll]=np.nan
    jnk=pd.to_timedelta(etime,unit='s')
    ztime=offset+jnk
    #
    profiletemp.append(temp[-1])
    profilesalt.append(salt[-1])
    profiledep.append(pres)

    #
    ptemp2.x_range=Range1d(ztime[0],ztime[-1])
    ptemp2.line(ztime,temp,line_width=2,color=colors[ctd-6])
    ptemp2.circle(ztime,temp,color=colors[ctd-6])
    #
    #print(salt[-10:-1])
    #print(ctds[ctd])
    #print(colors[ctd-6])
    psalt2.x_range=Range1d(ztime[0],ztime[-1])
    psalt2.line(ztime,salt,line_width=2,color=colors[ctd-6])
    psalt2.circle(ztime,salt,color=colors[ctd-6])
# the time-series plots are not used in this notebook...

In [5]:
# use the seawater routines to compute the density of seawater 
profileden=seawater.eos80.dens(profilesalt,profiletemp,profiledep)
# create sigma-T values
profileden=profileden-1000

In [6]:
# generate an output file for the plot
#output_file("m1_3axis_test.html")
# line below is for inline plots in a python notebook
output_notebook(notebook_type='jupyter')

Loading BokehJS ...

In [7]:
# compute the maximum and minumu depths for the y-axis limits
mind=min(profiledep)
maxd=max(profiledep)

In [10]:
# create the multi-axis profile plot.
# create tooltip for plot (unfortunately it only gets each individual point and not all at at same depth)
TOOLTIPCTD=[(("CTD data"),"$x")]
# create the starting figure from which the other axis will be added to.
# In this case we start with the temperature profile and color it blue and assiated axis
ctdfig=figure(plot_width=500,plot_height=800,x_axis_label='Temperature',y_axis_label='Depth',tooltips=TOOLTIPCTD)
ctdfig.xaxis.major_label_text_color='blue'
ctdfig.xaxis.axis_label_text_color='blue'
ctdfig.xaxis.major_tick_line_color='blue'
ctdfig.xaxis.minor_tick_line_color='blue'
ctdfig.xaxis.axis_line_color='blue'
ctdfig.line(profiletemp,profiledep,color='blue')
ctdfig.circle(profiletemp,profiledep,color='blue')
# set up randes for the plot (note to reverse the axis on depth reverse min and max on limits)
ctdfig.y_range=Range1d(float(maxd),float(mind))
ctdfig.x_range=Range1d(min(profiletemp),max(profiletemp))
# set up a dictionary for the extra axis we want to add
ctdfig.extra_x_ranges={'Salinity':Range1d(start=min(profilesalt), end=max(profilesalt)),'Density':Range1d(start=min(profileden),end=max(profileden))}
# plot data using the extra x-range
ctdfig.line(profilesalt,profiledep,color='green',x_range_name='Salinity')
ctdfig.circle(profilesalt,profiledep,color='green',x_range_name='Salinity')
# change the colors of axis, ticks, and labels.
ctdfig.add_layout(LinearAxis(x_range_name='Salinity',axis_label='Salinity',
                            axis_label_text_color='green',axis_line_color='green',
                            major_tick_line_color='green',minor_tick_line_color='green',
                            major_label_text_color='green'),'above')
ctdfig.line(profileden,profiledep,color='red',x_range_name='Density')
ctdfig.circle(profileden,profiledep,color='red',x_range_name='Density')
# might need more complex code to get a subscript on the sigma symbol...
ctdfig.add_layout(LinearAxis(x_range_name='Density',#axis_label='σ',
                            axis_label='σθ',
                            axis_label_text_color='red',axis_line_color='red',
                            major_tick_line_color='red',major_label_text_color='red',
                            minor_tick_line_color='red'),'above')
# actually generate the figure
show(ctdfig)


In [ ]:
#output_file("m1_3axis_test.html")

$\sigma_{\theta}$